In [1]:
from slimstampen.spacingmodel import SpacingModel, Fact, Response

In [2]:
m = SpacingModel()

In [3]:
# fact1 = Fact(fact_id = 1, context_1="ik ben en apple",context_2="jou bent en apple" answer = "apple")
# fact2 = Fact(2, "dog", "chien")
# fact3 = Fact(3, "cat", "chat")
# fact4 = Fact(4, "computer", "ordinateur")

# m.add_fact(fact1)
# m.add_fact(fact2)
# m.add_fact(fact3)
# m.add_fact(fact4)

In [4]:
# m.facts

In [5]:
# reading from a CSV
import pandas as pd
df = pd.read_table("./vocab.csv", names=["id","context_1", "context_2", "answer"], index_col=0, sep = ",", lineterminator='\n')
df.head(2)

,id,context_1,context_2,answer
1,1,je bent een eend,dat is een eend,duck
2,2,heb je een huis,ben je in een huis?,house


In [6]:
# adding all facts to spacing model
m = SpacingModel()
for row in df.itertuples():
    m.add_fact(Fact(fact_id = row.id,context_1= row.context_1, context_2 = row.context_2, answer=row.answer))
m.facts

[Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck'),
 Fact(fact_id=2, context_1='heb je een huis', context_2='ben je in een huis?', answer='house')]

In [7]:
next_fact, new = m.get_next_fact(current_time = 34000)
next_fact, new

(Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck'),
 True)

In [8]:
resp = Response(fact = next_fact, start_time = 34029, rt = 2207, correct = True)

m.register_response(resp)

In [9]:
m.responses

[Response(fact=Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck'), start_time=34029, rt=2207, correct=True)]

### Asking for more facts

When it's time to present the next trial, ask for another fact. With default parameter settings, the model chooses to immediately reinforce the fact that was just introduced, because it is likely to be forgotten otherwise. Note that this fact is now no longer marked as 'new'.

In [10]:
next_fact, new = m.get_next_fact(current_time = 38000)
next_fact, new

(Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck'),
 False)

Behind the scenes, the `get_next_fact` method has checked the expected activation of all facts slightly in the future (by default the model looks ahead 15 seconds). Since the activation of the first fact is predicted to be lower than the threshold of -0.8, it is selected for repetition. Note that facts that have not yet been studied have an activation of $-\infty$. 

In [11]:
for f in m.facts:
    print(f, m.calculate_activation(38000 + m.LOOKAHEAD_TIME, f))

Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck') -0.8828734492111224
Fact(fact_id=2, context_1='heb je een huis', context_2='ben je in een huis?', answer='house') -inf


Let's say this fact is again presented to the user and a correct response is made.

In [12]:
resp = Response(fact = next_fact, start_time = 38007, rt = 1890, correct = True)

m.register_response(resp)

The additional successful repetition means that the activation of this fact will now still be high enough in 15 seconds. 

In [13]:
for f in m.facts:
    print(f, m.calculate_activation(42000 + m.LOOKAHEAD_TIME, f))

Fact(fact_id=1, context_1='je bent een eend', context_2='dat is een eend', answer='duck') -0.4389281810299998
Fact(fact_id=2, context_1='heb je een huis', context_2='ben je in een huis?', answer='house') -inf


Sure enough, when we ask for a new fact, we get a different one:

In [14]:
next_fact, new = m.get_next_fact(current_time = 42000)
next_fact, new

(Fact(fact_id=2, context_1='heb je een huis', context_2='ben je in een huis?', answer='house'),
 True)

### Estimated *rate of forgetting* ($\alpha$)

A fact's rate of forgetting ($\alpha$) always starts at 0.3. Once at least 3 responses have been recorded, this value is adjusted up (if the fact is difficult to remember) or down (if it's easy to remember).

The `get_rate_of_forgetting` method returns the estimated rate of forgetting for a fact at a specified time, given the responses that were made before that time.

We can confirm that the rate of forgetting estimate for the first fact is indeed 0.3 before any responses have been logged:  

In [15]:
print('At t=0: {}'.format(m.get_rate_of_forgetting(0, fact=next_fact)))

At t=0: 0.3


If we add some more responses for `fact1` we can see how adjustments in the estimated rate of forgetting happen (notice that adjustment only starts after response #3). In this case, the observed response times are lower than expected, so the rate of forgetting estimate is adjusted downwards to about 0.20.

In [16]:
print('After 2 responses: {}'.format(m.get_rate_of_forgetting(50000, next_fact)))

resp = Response(fact = next_fact, start_time = 50000, rt = 1200, correct = True)
m.register_response(resp)

print('After 3 responses: {}'.format(m.get_rate_of_forgetting(60000, next_fact)))

resp = Response(fact = next_fact, start_time = 60000, rt = 1100, correct = True)
m.register_response(resp)

print('After 4 responses: {}'.format(m.get_rate_of_forgetting(70000, next_fact)))

resp = Response(fact = next_fact, start_time = 70000, rt = 1000, correct = True)
m.register_response(resp)

print('After 5 responses: {}'.format(m.get_rate_of_forgetting(80000, next_fact)))


After 2 responses: 0.3
After 3 responses: 0.3
After 4 responses: 0.3
After 5 responses: 0.250390625


### Exporting the response data

The method `export_data` provides a simple way of saving the response data and the model estimates. It also returns a copy of the data in case you want to do more with it.

The column `alpha` contains the estimated rate of forgetting *after* the trial.

In [17]:
m.export_data("data.csv")

,start_time,rt,correct,fact_id,context_1,context_2,answer,alpha,reading_time
trial,,,,,,,,,
1,34029,2207,True,1,je bent een eend,dat is een eend,duck,0.300000,300
2,38007,1890,True,1,je bent een eend,dat is een eend,duck,0.300000,300
3,50000,1200,True,2,heb je een huis,ben je in een huis?,house,0.300000,300
4,60000,1100,True,2,heb je een huis,ben je in een huis?,house,0.300000,300
5,70000,1000,True,2,heb je een huis,ben je in een huis?,house,0.250391,300
